In [12]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# import tweepy
import random

In [10]:
# example url access
data_url = "https://raw.githubusercontent.com/echen102/COVID-19-TweetIDs/master/2021-10/coronavirus-tweet-id-2021-10-01-00.txt"
tweet_data = pd.read_csv(data_url, header=None, sep='\s+')
# tweet_data.rename(columns={0: 'tweet_ids'}, inplace=True)
# tweet_data.head()
tweet_data.iloc[[0,3]][0]

0    1443727336915054593
3    1443727337225273344
Name: 0, dtype: int64

In [18]:
# example url access
number_scraped = 100000/0.6359 #number to be scraped taking into account number of expected english tweets we'll find
number_per_month = number_scraped/23 # 23 months of covid
number_per_day = lambda month: number_per_month/31 if month in [1, 3, 5, 7, 8, 10, 12] else number_per_month/30
number_per_hour = lambda month: int(number_per_day(month)/16) # least activity is midnight to 8am: https://smallbusiness.chron.com/peak-times-twitter-activity-62864.html
years = ['2020', '2021']
months = list(range(1, 13))
days = lambda month: list(range(1,32)) if month in [1, 3, 5, 7, 8, 10, 12] else list(range(1,31))
hours = list(range(8, 24))
banned = [('03', '02', 0, 23), ('02', '23', 0, 23), ('02', '22', 21, 23), ('02', '01', 4, 9)]
base_url = "https://raw.githubusercontent.com/echen102/COVID-19-TweetIDs/master/" #2021-10/coronavirus-tweet-id-2021-10-01-00.txt
with open('tweet_ids.txt', 'w') as f:
    for year in years:
        for month in months:
            month = str(month) if month >= 10 else '0' + str(month)
            print('Writing Month: ' + month)
            for day in days(month):
                day = str(day) if day >= 10 else '0' + str(day)
                if year == '2020' and month == '01' and day < '22':
                    continue
                for hour in hours:
                    if len(banned) != 0 and month > banned[-1][0] or (month == banned[-1][0] and day > banned[-1][1]):
                        banned = banned[:-1]
                    if len(banned) != 0 and month == banned[-1][0] and day == banned[-1][1] and year == '2020' and hour >= banned[-1][2] and hour <= banned[-1][3]:
                        continue
                    hour = str(hour) if hour >= 10 else '0' + str(hour)
                    url = base_url + year + '-' + month + '/coronavirus-tweet-id-' + year + '-' + month + '-' + day + '-' + hour + '.txt'
                    tweet_data = pd.read_csv(data_url, header=None, sep='\s+')
                    indexes = random.sample(range(0, len(tweet_data)), number_per_hour(int(month)))
                    f.write('\n'.join([str(x) for x in list(tweet_data.iloc[indexes][0])]))
                    f.write('\n')
f.close()
                
                
                
                
                

KeyboardInterrupt: 

In [3]:
tweet_data['tweet_ids'].dtype

dtype('int64')

In [ ]:
# https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id
# https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1
# https://developer.twitter.com/en/portal/dashboard
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
# https://twitter.com/edent/status/

In [4]:
consumer_key = 'cSuIh662am6YtzuJNT1RWnzqF'
consumer_secret = 'V7uyDK3o3OLljPWMzQ9pKzGPK8tODHVFkHBgwDuEvf9c3g1eaX'
access_token = '2577558920-yKXNYAqsm3agfTwCuVLnDHQrrSfeIptFUIkqcID'
access_token_secret = '3GSupBB3egr7wzFY3QN2uVwGy3q38fkTaavNuzS1X4Ew0'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [9]:
try:
    tweet = api.get_status(tweet_data['tweet_ids'][2], tweet_mode = "extended")
    print(tweet.__dict__)
    print(tweet_data['tweet_ids'][2])
except:
    print("tweet_id not avail:", tweet_data['tweet_ids'][2])

{'_api': <tweepy.api.API object at 0x7f7f7478e850>, '_json': {'created_at': 'Thu Sep 30 23:59:54 +0000 2021', 'id': 1443727336961032193, 'id_str': '1443727336961032193', 'full_text': '"Comparison of COVID‐19 studies registered in the clinical trial platforms: A research ethics analysis perspective" https://t.co/Y7weqIW5aw #Bioethics #ResearchEthics #ClinicalTrials', 'truncated': False, 'display_text_range': [0, 182], 'entities': {'hashtags': [{'text': 'Bioethics', 'indices': [140, 150]}, {'text': 'ResearchEthics', 'indices': [151, 166]}, {'text': 'ClinicalTrials', 'indices': [167, 182]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Y7weqIW5aw', 'expanded_url': 'https://onlinelibrary.wiley.com/doi/full/10.1111/dewb.12333#.YVZPXX8hFDs.twitter', 'display_url': 'onlinelibrary.wiley.com/doi/full/10.11…', 'indices': [116, 139]}]}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_s

In [22]:
tweet_data['text'] = np.nan
tweet_data['urls'] = np.nan
tweet_data['favorites'] = np.nan
tweet_data['retweets'] = np.nan
for index, tweet in enumerate(tweet_data['tweet_ids'][:10]):
    try:
        obj = api.get_status(tweet, tweet_mode = "extended")
        try:
            tweet_data['text'][index] = obj.full_text
        except:
            print('error retrieving text')
        try: 
            tweet_data['urls'][index] = [url.get('url') for url in obj._json.get('entities').get('urls')]
        except:
            print('error retrieving urls')
        try: 
            tweet_data['favorites'][index] = obj._json.get('retweeted_status').get('favorite_count')
        except:
            print('error retrieving likes')
        try: 
            tweet_data['retweets'][index] = obj.retweet_count
        except:
            print('error retrieving retweets')
    except:
        pass
    
tweet_data

    

<ipython-input-22-44f3a7691e90>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_data['text'][index] = obj.full_text
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-22-44f3a7691e90>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_data['urls'][index] = [url.get('

error retrieving likes
error retrieving likes
error retrieving likes


,tweet_ids,text,urls,favorites,retweets
0,1443727336915054593,"RT @RT_com: YouTube bans ALL anti-vax videos, ...",[https://t.co/XKRQnzvWeh],82.0,50.0
1,1443727337087021056,NaN,NaN,NaN,NaN
2,1443727336961032193,"""Comparison of COVID‐19 studies registered in ...",[https://t.co/Y7weqIW5aw],NaN,0.0
3,1443727337225273344,"RT @VicGovDH: Reported yesterday: 1,143 new lo...",[],1349.0,380.0
4,1443727337229594629,RT @KATUNews: United Airlines says the declini...,[],259.0,28.0
...,...,...,...,...,...
90992,1443742435536998404,NaN,NaN,NaN,NaN
90993,1443742435545427971,NaN,NaN,NaN,NaN
90994,1443742435671109633,NaN,NaN,NaN,NaN
90995,1443742435964854276,NaN,NaN,NaN,NaN


In [8]:
api.get_status(tweet, tweet_mode = "extended").__dict__

{'_api': <tweepy.api.API at 0x7f7f7478e850>,
 '_json': {'created_at': 'Thu Sep 30 23:59:55 +0000 2021',
  'id': 1443727337439305730,
  'id_str': '1443727337439305730',
  'full_text': 'RT @revistaoeste: Luciano Hang decide processar senadores da CPI da Covid https://t.co/1owhxzPyNL',
  'truncated': False,
  'display_text_range': [0, 97],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'revistaoeste',
     'name': 'Revista Oeste',
     'id': 1222939896349970432,
     'id_str': '1222939896349970432',
     'indices': [3, 16]}],
   'urls': [{'url': 'https://t.co/1owhxzPyNL',
     'expanded_url': 'https://revistaoeste.com/politica/luciano-hang-decide-processar-senadores-da-cpi-da-covid/',
     'display_url': 'revistaoeste.com/politica/lucia…',
     'indices': [74, 97]}]},
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_